In [29]:
import pandas as pd
import numpy as np

#Acquiring data from Wine Quality Tables
redTable = pd.read_csv("/content/winequality-red.csv", header = None, sep = ";")
whiteTable = pd.read_csv("/content/winequality-white (1).csv", header = None, sep = ";")
randomW = np.random.randint(0,1600,size=5)
randomR = np.random.randint(0,1600,size=5)

#Assigning values for the wines that are sold at the shop
wineName = ["Boulangerie", "Chanson", "Grisaille", "Mademoiselle", "Vivant", "Chaussures", "Mignon", "Magnifique", "Alunissage", "Chatoyer"]
wineStock = np.random.randint(6,12,size=10) #how much wine is in stock
winePrice = np.random.randint(10,40,size=10)
wineType = [0,0,0,0,0,0,0,0,0,0] #if 0: red wine || if 1: white wine
wineSugar = [0,0,0,0,0,0,0,0,0,0]
winePriceCopy = winePrice

#Getting sugar and wine types from the data tables
for i in range(10):
  if(i < 5):
    wineType[i] = 0
    wineSugar[i] = float(redTable[3][randomR[i]])
  else:
    wineType[i] = 1
    wineSugar[i] = float(whiteTable[3][randomW[i-5]])

#Deciding which wine each 100 customers will buy during trial day
#Also: choosing one wine that is more popular than the rest (Vivant) and giving it a higher probability to be bought
wineProb1 = [9,9,9,9,19,9,9,9,9,9]
wineProb = [9,0,0,0,0,0,0,0,0,0]
for i in range(1,10):
  wineProb[i] = wineProb[i-1] + wineProb1[i]
customerTest = np.random.randint(0,100,size=100)
for i in range(100):
  temp = np.random.randint(1,100)
  if(0 < temp & temp <= wineProb[0]):
    customerTest[i] = 0
  if(wineProb[0] < temp & temp <= wineProb[1]):
    customerTest[i] = 1
  if(wineProb[1] < temp & temp <= wineProb[2]):
    customerTest[i] = 2
  if(wineProb[2] < temp & temp <= wineProb[3]):
    customerTest[i] = 3
  if(wineProb[3] < temp & temp <= wineProb[4]):
    customerTest[i] = 4
  if(wineProb[4] < temp & temp <= wineProb[5]):
    customerTest[i] = 5
  if(wineProb[5] < temp & temp <= wineProb[6]):
    customerTest[i] = 6
  if(wineProb[6] < temp & temp <= wineProb[7]):
    customerTest[i] = 7
  if(wineProb[7] < temp & temp <= wineProb[8]):
    customerTest[i] = 8
  if(wineProb[8] < temp & temp <= wineProb[9]):
    customerTest[i] = 9

#Metrics for store, including keeping track of how much wine they sell of each bottle and how many people cannot buy the wine they want
storeStock = np.copy(wineStock)
numWine = [0,0,0,0,0,0,0,0,0,0]
turnaways = [0,0,0,0,0,0,0,0,0,0]
revenue = 0
loss = 0
typeAmt = [0,0]
sugarAvg = 0

#Run through the 
for i in range(100):
  if(storeStock[customerTest[i]] > 0):
    storeStock[customerTest[i]] = storeStock[customerTest[i]] - 1
    revenue = revenue + winePrice[customerTest[i]]
  else:
    turnaways[customerTest[i]] = turnaways[customerTest[i]] + 1
    loss = loss + winePrice[customerTest[i]]
  numWine[customerTest[i]] = numWine[customerTest[i]] + 1
  typeAmt[wineType[customerTest[i]]] = typeAmt[wineType[customerTest[i]]] + 1
  sugarAvg = wineSugar[customerTest[i]] + sugarAvg

sugarAvg = sugarAvg/100
revenueTemp = revenue
lossTemp = loss
redTemp = typeAmt[0]
whiteTemp = typeAmt[1]

#Membership Program Kicks In, following is adjusted every 14 trials, emulating 2 weeks

#Adjust stock to make sure people are able to buy wines they like, and less of the wines that dont sell as well
#People will be 1% more likely to buy the most popular wine and 3% more likely to buy the other newly discounted wines
#Adjusting stock of the most popular wines by allocating 3 bottles to Vivant and 2 to the newly discounted wines

#Run through "n" trials and determine revenue/loss growth or decrease
#Introduce a discound hold if the negative or positive change in loss is high (100 dollars)
n = 100
revenueArr = np.random.randint(0,1,size=n)
lossArr = np.random.randint(0,1,size=n)
redArr = np.random.randint(0,1,size=n)
whiteArr = np.random.randint(0,1,size=n)
sugarArr = np.random.randint(0,1,size=n)
for b in range(n):
  if(b%14 == 0):
    maxW = 0
    maxR = 0
    winePrice = winePriceCopy
    for j in range(2):
      for i in range(10):
        if j == 0:
          #Finding most popular red and white wines (not including Viviant)
          if(i < 5):
            if(storeStock[maxR] < storeStock[i]):
              maxR = i
          if(i > 4):
            if(storeStock[maxW] < storeStock[i]):
              maxW = i
        else: 
          #Adjusting probabilities and wines
          if(b == 0):
            #first installation of the discount process
            if(i == 4):
              #most popular wine gets the 5% price discount, 1% probability boost, and 3 extra bottles in stock
              winePrice[i] = winePrice[i] - (winePrice[i] * .05)
              wineProb1[i] = wineProb1[i] + 1
              wineStock[i] = wineStock[i] + 3 
            if (i == maxR or i == maxW):
              #next two most popular wines gets the 5% price discount, 3% probability boost, and 2 extra bottles in stock
              winePrice[i] = winePrice[i] - (winePrice[i] * .05)
              wineProb1[i] = wineProb1[i] + 3
              wineStock[i] = wineStock[i] + 2   
            else:
              #other wines drop in shopping probability by 1% and lose one bottle of stock
              wineProb1[i] = wineProb1[i] - 1
              wineStock[i] = wineStock[i] - 1

          else:
            #runs after first installation of discounts
            if (i == maxR or i == maxW):
              #two most popular wines gets the 5% price discount, 2% probability boost, and 2 extra bottles in stock
              winePrice[i] = winePrice[i] - (winePrice[i] * .05)
              wineProb1[i] = wineProb1[i] + 2
              wineStock[i] = wineStock[i] + 2   
            else:
              #other wines drop in shopping probability by .5% and two randomly lose one bottle of stock
              wineProb1[i] = wineProb1[i] - .5
              wineLoss = np.random.randint(0,7,size=2)
              if(i == wineLoss[0] or i == wineLoss[1]):
                wineStock[i] = wineStock[i] - 1

  #Assign wine purchasing probabilities
  wineProb = [wineProb1[0],0,0,0,0,0,0,0,0,0]
  for i in range(1,10):
    wineProb[i] = wineProb[i-1] + wineProb1[i]
  customerTest = np.random.randint(0,1,size=100)
  for i in range(100):
    temp = np.random.randint(1,100)
    if(0 < temp and temp <= wineProb[0]):
      customerTest[i] = 0
    elif(wineProb[0] < temp and temp <= wineProb[1]):
      customerTest[i] = 1
    elif(wineProb[1] < temp and temp <= wineProb[2]):
      customerTest[i] = 2
    elif(wineProb[2] < temp and temp <= wineProb[3]):
      customerTest[i] = 3
    elif(wineProb[3] < temp and temp <= wineProb[4]):
      customerTest[i] = 4
    elif(wineProb[4] < temp and temp <= wineProb[5]):
      customerTest[i] = 5
    elif(wineProb[5] < temp and temp <= wineProb[6]):
      customerTest[i] = 6
    elif(wineProb[6] < temp and temp <= wineProb[7]):
      customerTest[i] = 7
    elif(wineProb[7] < temp and temp <= wineProb[8]):
      customerTest[i] = 8
    elif(wineProb[8] < temp and temp <= wineProb[9]):
      customerTest[i] = 9
  storeStock = np.copy(wineStock)
  numWine = [0,0,0,0,0,0,0,0,0,0]
  turnaways = [0,0,0,0,0,0,0,0,0,0]
  revenue = 0
  loss = 0
  typeAmt = [0,0]
  sugarAvg = 0

  #Run a period of 100 members after this trial period
  for i in range(100):
    if(storeStock[customerTest[i]] > 0):
      storeStock[customerTest[i]] = storeStock[customerTest[i]] - 1
      revenue = revenue + winePrice[customerTest[i]]
    else:
      turnaways[customerTest[i]] = turnaways[customerTest[i]] + 1
      loss = loss + winePrice[customerTest[i]]
    numWine[customerTest[i]] = numWine[customerTest[i]] + 1
    typeAmt[wineType[customerTest[i]]] = typeAmt[wineType[customerTest[i]]] + 1
    sugarAvg = wineSugar[customerTest[i]] + sugarAvg
  #Keep track of metrics during the "n" number of trials
  revenueArr[b] = revenue
  lossArr[b] = loss
  redArr[b] = typeAmt[0]
  whiteArr[b] = typeAmt[1]
  sugarArr[b] = sugarAvg/100

print("Revenue Comparison")
print("Original Revenue from First Trial: ", revenueTemp)
print("New Revenues: ", revenueArr)
print()
print("Loss Comparison")
print("Original Loss from First Trial: ",lossTemp)
print("New Loss: ",lossArr)
print()
print("Amount of Reds and Whites bought")
print("Reds: ", redArr)
print("Whites: ", whiteArr)
print()
print("Average Sugar Content")
print("Average Sugar: ", sugarArr)

Revenue Comparison
Original Revenue from First Trial:  1605
New Revenues:  [1562 1686 1657 1750 1628 1653 1762 1627 1607 1590 1704 1708 1637 1495
 1646 1644 1641 1657 1716 1658 1617 1486 1515 1493 1607 1464 1596 1543
 1528 1519 1556 1511 1582 1606 1611 1659 1652 1626 1674 1671 1591 1563
 1606 1544 1594 1647 1791 1682 1803 1647 1715 1631 1612 1686 1756 1739
 1778 1711 1587 1589 1555 1694 1635 1747 1679 1578 1659 1628 1762 1641
 1526 1777 1524 1697 1573 1669 1582 1839 1729 1667 1618 1649 1595 1507
 1637 1521 1625 1653 1626 1465 1560 1621 1814 1525 1674 1687 1637 1740
 1757 1676]

Loss Comparison
Original Loss from First Trial:  671
New Loss:  [514 528 473 367 538 527 408 509 602 482 554 414 490 707 474 524 485 517
 410 542 472 640 637 618 533 540 473 499 537 481 453 507 456 426 437 368
 438 464 431 466 413 429 367 485 524 449 348 336 317 381 327 419 384 355
 324 349 253 301 379 496 364 295 428 280 333 469 384 351 347 323 499 224
 381 320 451 266 367 212 304 278 358 279 407 429 400 393 29